In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv

## Torch model

In [4]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [5]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [6]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# MuxConv

In [7]:
from time import time

In [19]:
muxed_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [20]:
ctxt, outs0 = muxed_model.forward_early(img_tensor)
print(ctxt[:64])        
# Basic blocks
ctxt, outs1 = muxed_model.forward_bb(model.layer1[0], ctxt, outs0)
print(ctxt[:64])        
ctxt, outs2 = muxed_model.forward_bb(model.layer2[0], ctxt, outs1)
print(ctxt[:64])        
ctxt, outs3 = muxed_model.forward_bb(model.layer3[0], ctxt, outs2)
print(ctxt[:64])        

[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
1.2639 s
[ 1.23083757e+00  4.86967574e-01  2.75344729e-01  3.29138183e-01
  4.28335613e-01  3.70344855e-01  4.40048646e-01  4.22215130e-01
  3.75639522e-01  4.12476729e-01  3.55572611e-01  1.82748623e-01
  2.23403126e-12 -1.25110832e-12 -3.93159852e-12  2.29966678e-02
  7.94883424e-02  1.53448636e-01  2.66038135e-01  4.51326009e-01
  4.74084721e-01  4.09967894e-01  3.81854797e-01  3.23575674e-01
  3.89218624e-01  4.11176149e-01  2.51731082e-01  2.53241743e-01
  3.68447519e-01  2.82682075e-01  3.29150118e-01  2.50681995e-01
  1.44370664e+00  1.43021865e-01  4.49264536e-02  5.36310604e-02
  9.95615098e-02 -6.66724757e-04  5.28738508e-02  1.67052255e-01
 -6.62974982e-04  6.76758060e-12  5.58868822e-12  8.63362800e-11
  1.20594992e-10  1.16509341e-10  1.52629003e-10 -8.16614189e-11
  6.68249524e-11  1.00328779e-10 -3.75426353e-12  1.81684809e-01
  2.61866027e-01  8.90501600e-02  4.47203087e-02  4.56751499e-02
  1.33089175e-01  8.685

In [7]:
t0 = time()
result = muxed_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
1.3540 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
5.3013 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
5.3148 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
10.2857 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
4.8755 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
1.4668 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
9.2939 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =( 8, 8,64, 4, 4,  8)
4.6312 s
[MultParConvBN] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
1.4283 s
43.9756 s


In [19]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8151,  -3.1035,  -2.9026,  -9.4615,   4.3280,  -6.9753,
         -3.5617,  -8.8841,  -7.5631], dtype=torch.float64)